In [ ]:
from __future__ import print_function
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
# number 1 to 10 data
mnist = input_data.read_data_sets('MNIST_data', one_hot=True)

def xavier_init(size):
    in_dim = size[0]
    xavier_stddev = 1. / tf.sqrt(in_dim / 2.)
    return tf.random_normal(shape=size, stddev=xavier_stddev)

def Get_trainData():
    train_xs = mnist.train.images
    train_ys = mnist.train.labels

    #print('train_xs',train_xs)
    #print('train_ys',train_ys)

    img_xs = []
    label_ys = []

    for t1 in range(50):
        #print('t1',t1)
        t1val = t1*1000
        test_img = train_xs[t1val:t1val+1000] 
        test_ys = train_ys[t1val:t1val+1000] 
        y_pre = sess.run(prediction, feed_dict={xs: test_img,ys:test_ys, keep_prob: 1})
        argmaxResu = tf.argmax(test_ys,1)
        argmaxCheck =tf.argmax(y_pre,1)
        resu = sess.run(argmaxResu,feed_dict={})
        #print('resu',resu)
        runresult = sess.run(argmaxCheck,feed_dict={})
        #print('Run Result',runresult)
        correct_prediction = tf.equal(argmaxResu, argmaxCheck)
        checkResu = sess.run(correct_prediction,feed_dict={})
        #print('correct_prediction',checkResu)
        #print('shape',checkResu.shape[0])
        t2_resu = checkResu.shape[0]
        for t2 in range(t2_resu):
            #print('t2',t2)
            if (checkResu[t2]==False):
                img_xs.append(test_img[t2])
                label_ys.append(test_ys[t2])
        #img_xs = tf.convert_to_tensor(img_xs)
        #label_ys = tf.convert_to_tensor(label_ys)
        #img_xs = tf.cast(img_xs, tf.float32)
        #label_ys = tf.cast(label_ys, tf.float32)
    #print('img :',img_xs.shape,img_xs.shape)
    #print('label :',label_ys.shape,label_ys.shape)
    return img_xs,label_ys

def NextData(img_xs,label_ys):
    img_xs1 = img_xs[0:100]
    label_ys1= label_ys[0:100]
    img_xs = img_xs[100:]
    label_ys= label_ys[100:]
    img_xs.append(img_xs1)
    label_ys.append(label_ys1)
    return img_xs,label_ys

def trainData():
    batch_xs, batch_ys =Get_trainData()
#sess.run(accuracy, feed_dict={xs: v_xs, ys: v_ys, keep_prob: 1})

    for i in range(200):        
        sess.run(train_step, feed_dict={xs: batch_xs, ys: batch_ys, keep_prob: 0.5})
        curracc = compute_accuracy(
                    mnist.test.images[k:k+1000], mnist.test.labels[k:k+1000])
        y_pre = sess.run(prediction, feed_dict={xs: batch_xs, ys: batch_ys})    
    
    
        
        if i % 10 == 0:
            k= int(i/50)%3*1000
        
            print(compute_accuracy(
                    mnist.test.images[k:k+1000], mnist.test.labels[k:k+1000]))
            print('TImes',i)
            k= int(i/10)%9*1000
        
            print(compute_accuracy(
                    mnist.test.images[k:k+1000], mnist.test.labels[k:k+1000]))
    return
def compute_accuracy(v_xs, v_ys):

    global prediction1
    y_pre = sess.run(prediction, feed_dict={xs: v_xs, keep_prob: 1})
    correct_prediction = tf.equal(tf.argmax(y_pre,1), tf.argmax(v_ys,1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    result = sess.run(accuracy, feed_dict={xs: v_xs, ys: v_ys, keep_prob: 1})
    return result


def compute_accuracy1(v_xs, v_ys):
    
    global prediction1
    y_pre = sess.run(prediction1, feed_dict={xs: v_xs, keep_prob: 1})
    correct_prediction = tf.equal(tf.argmax(y_pre,1), tf.argmax(v_ys,1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    result = sess.run(accuracy, feed_dict={xs: v_xs, ys: v_ys, keep_prob: 1})
    return result

def weight_variable2(shape):

    initial = tf.truncated_normal(shape, stddev=0.7)
    return tf.Variable(initial)

def weight_variable(shape):

    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial)

def bias_variable(shape):

    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)

def conv2d(x, W):

    # stride [1, x_movement, y_movement, 1]
    # Must have strides[0] = strides[3] = 1
    return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')

def max_pool_2x2(x):

    # stride [1, x_movement, y_movement, 1]
    return tf.nn.max_pool(x, ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME')

# define placeholder for inputs to network

h_dim = 120
checkpoint_dir ='./'

restorePara = True

testPara = False

xs = tf.placeholder(tf.float32, [None, 784])/255.   # 28x28
ys = tf.placeholder(tf.float32, [None, 10])
keep_prob = tf.placeholder(tf.float32)
with tf.name_scope('Reshape'):
    x_image = tf.reshape(xs, [-1, 28, 28, 1])
# print(x_image.shape)  # [n_samples, 28,28,1]

## conv1 layer ##

with tf.name_scope('convolution1'):
    with tf.name_scope('W_conv1'):
        W_conv1 = weight_variable([5,5, 1,32]) # patch 5x5, in size 1, out size 32
    with tf.name_scope('b_conv1'):
        b_conv1 = bias_variable([32])
    with tf.name_scope('h_conv1'):
        h_conv1 = tf.nn.leaky_relu(conv2d(x_image, W_conv1) + b_conv1,0.2) # output size 28x28x32
    with tf.name_scope('Max_pooling1'):
        h_pool1 = max_pool_2x2(h_conv1)                                         # output size 14x14x32

## conv2 layer ##
#with tf.name_scope('conv2 layer'):
with tf.name_scope('convolution2'):
    with tf.name_scope('W_conv2'):
        W_conv2 = weight_variable([5,5, 32, 128]) # patch 5x5, in size 32, out size 64
    with tf.name_scope('b_conv2'):
        b_conv2 = bias_variable([128])
    with tf.name_scope('h_conv2'):
        h_conv2 = tf.nn.leaky_relu(conv2d(h_pool1, W_conv2) + b_conv2,0.2) # output size 14x14x64
                                           # output size 7x7x128

## conv2 layer ##
#with tf.name_scope('conv2 layer'):
with tf.name_scope('convolution3'):
    with tf.name_scope('W_conv3'):
        W_conv3 = weight_variable([3,3, 128, 256]) # patch 5x5, in size 32, out size 64
    with tf.name_scope('b_conv3'):
        b_conv3 = bias_variable([256])
    with tf.name_scope('h_conv3'):
        h_conv3 = tf.nn.leaky_relu(conv2d(h_conv2, W_conv3) + b_conv3,0.2)               # output size 7x7x128   
    with tf.name_scope('Max_pooling2'):
        h_pool3 = max_pool_2x2(h_conv3) 

with tf.name_scope('convolution1'):
    with tf.name_scope('W_conv1'):
        W_conv1_1 = weight_variable([5,5, 1,32]) # patch 5x5, in size 1, out size 32
    with tf.name_scope('b_conv1'):
        b_conv1_1 = bias_variable([32])
    with tf.name_scope('h_conv1'):
        h_conv1_1 =  tf.nn.leaky_relu(conv2d(x_image, W_conv1_1) + b_conv1_1,0.2) # output size 28x28x32
with tf.name_scope('Max_pooling1'):
        h_pool1_1 = max_pool_2x2(h_conv1_1)                                         # output size 14x14x32

## conv2 layer ##
#with tf.name_scope('conv2 layer'):
with tf.name_scope('convolution2'):
    with tf.name_scope('W_conv2'):
        W_conv2_1 = weight_variable([5,5, 32, 512]) # patch 5x5, in size 32, out size 64
    with tf.name_scope('b_conv2'):
        b_conv2_1 = bias_variable([512])
    with tf.name_scope('h_conv2'):
        h_conv2_1 =  tf.nn.leaky_relu(conv2d(h_pool1_1, W_conv2_1) + b_conv2_1,0.2) # output size 14x14x64
    with tf.name_scope('Max_pooling2'):
        h_pool2_1 = max_pool_2x2(h_conv2_1)                                         # output size 7x7x128

## conv2 layer ##
#with tf.name_scope('conv2 layer'):
with tf.name_scope('convolution3'):
    with tf.name_scope('W_conv3'):
        W_conv3_1 = weight_variable([5,5, 512, 128]) # patch 5x5, in size 32, out size 64
    with tf.name_scope('b_conv3'):
        b_conv3_1 = bias_variable([128])
    with tf.name_scope('h_conv3'):
        h_conv3_1 =  tf.nn.leaky_relu(conv2d(h_pool2_1, W_conv3_1) + b_conv3_1,0.2)               # output size 7x7x128   

with tf.name_scope('convolution5'):
    with tf.name_scope('W_conv5'):
        W_conv5_1 = weight_variable([5,5, 128, 256]) # patch 5x5, in size 32, out size 64
    with tf.name_scope('b_conv5'):
        b_conv5_1 = bias_variable([256])
    with tf.name_scope('h_conv5'):              
        h_conv5_1 = tf.nn.relu(conv2d(h_conv3_1, W_conv5_1) + b_conv5_1)               # output size 4x4x128 
    with tf.name_scope('Max_pooling5'):
        h_pool5_1 = max_pool_2x2(h_conv5_1)                                         # output size 7x7x128
        print('h_pool5',h_pool5_1) 
        
        
        
        
with tf.name_scope('convolution5'):
    with tf.name_scope('W_conv5'):
        W_conv5 = weight_variable([3,3, 256, 256]) # patch 5x5, in size 32, out size 64
    with tf.name_scope('b_conv5'):
        b_conv5 = bias_variable([256])
    with tf.name_scope('h_conv5'):       
        h_conv5 = tf.nn.relu(conv2d(h_pool3, W_conv5) + b_conv5)               # output size 4x4x128 
    with tf.name_scope('Max_pooling5'):
        h_pool5 = max_pool_2x2(h_conv5)                                         # output size 7x7x128
        print('h_pool5',h_pool5)


#with tf.name_scope('fc1 layer'):
#with tf.name_scope('Fully_Connection_1'):
#    with tf.name_scope('W_fc1'):
#        W_fc1 = weight_variable([7*7*64, 1024])
#    with tf.name_scope('b_fc1'):
#        b_fc1 = bias_variable([1024])
#    with tf.name_scope('h_pool2_flat'):
# [n_samples, 7, 7, 64] ->> [n_samples, 7*7*64]
#        h_pool2_flat = tf.reshape(h_pool2, [-1, 7*7*64])
#    with tf.name_scope('h_fc1'):
#        h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)
#    with tf.name_scope('h_fc1_drop'):
#        h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)

## fc2 layer ##
#with tf.name_scope('fc2 layer'):
with tf.name_scope('Fully_Connection_2'):
#    with tf.name_scope('W_fc1'):
#        W_fc1 = weight_variable([7*7*64, 10])
#    with tf.name_scope('b_fc1'):
#        b_fc1 = bias_variable([10])
    with tf.name_scope('h_pool2_flat'): 
        h_pool2_flat = tf.reshape(h_pool5, [-1, 4*4*256])
    with tf.name_scope('h_pool2_flat_1'): 
        h_pool2_flat_1 = tf.reshape(h_pool5_1, [-1, 4*4*256])
        
        #hidden = tf.concat(axis=1, values=[h_pool2_flat,h_pool2_flat_1])
      
        #print('hidden',hidden)
        
        #h_pool2_flat = h_pool2_flat+h_pool2_flat_1
    with tf.name_scope('W_fc2'):
        W_fc2 = weight_variable([4*4*256, 10])      
    with tf.name_scope('W_fc2_1'):
        W_fc1 = weight_variable([4*4*256, 10])      
      
    with tf.name_scope('b_fc2'):
        b_fc2 = bias_variable([10])  
    with tf.name_scope('b_fc2_1'):
        b_fc1 = bias_variable([10])  
      
    
    with tf.name_scope('b_check'):
        D_W1 = tf.Variable(xavier_init([784, h_dim]))
    with tf.name_scope('b_checkfc'):
        D_b1 = tf.Variable(tf.zeros(shape=[h_dim]))
    
    with tf.name_scope('b_check2'):
        D_W2 = tf.Variable(xavier_init([h_dim, 90]))
    with tf.name_scope('b_checkfc2'):
        D_b2 = tf.Variable(tf.zeros(shape=[90]))
        
    with tf.name_scope('b_check3'):
        D_W3 = tf.Variable(xavier_init([90, 10]))
    with tf.name_scope('b_checkfc3'):
        D_b3 = tf.Variable(tf.zeros(shape=[10]))
        
        
    
    
    
    
#with tf.name_scope('resu'):
w_fc = tf.Variable(tf.ones(shape=[30,10]))
#weight_variable2([30,10]) 
#tf.Variable(tf.ones(shape=[30,10]))

#    #weight_variable2([30,10])  

print('wfc',w_fc)
    #with tf.name_scope('b_resu'):
b_fc = bias_variable([10])  
    
    #with tf.name_scope('prediction'):      
x_image2 = tf.reshape(xs, [-1, 784])
D_h1 = tf.nn.leaky_relu(tf.matmul(x_image2, D_W1) + D_b1,0.2)   
#D_h1 = tf.nn.tanh(D_h1)
D_h2 = tf.nn.leaky_relu(tf.matmul(D_h1, D_W2) + D_b2,0.2)
D_h2 = tf.nn.tanh(D_h2)
D_logit = tf.matmul(D_h2, D_W3) + D_b3
D_logit = tf.nn.leaky_relu(D_logit,0.2)
D_logitResu = tf.nn.tanh(D_logit)     
        #D_logit = bacthnorm(D_logit,'logit')
        
        #prediction = tf.nn.softmax(tf.add(tf.matmul(h_pool2_flat, W_fc2)+b_fc2,tf.matmul(h_pool2_flat_1, W_fc1)+b_fc1 ))
hidden1 = tf.nn.leaky_relu(tf.matmul(h_pool2_flat, W_fc2)+b_fc2,0.2)
hidden2 = tf.nn.leaky_relu(tf.matmul(h_pool2_flat_1, W_fc1)+b_fc1,0.2)        
        
prediction1= tf.nn.softmax(D_logitResu)
prediction2= tf.nn.softmax(hidden1)
prediction3= tf.nn.softmax(hidden2)        
        
print('hidden1',hidden1)
hidden1 = tf.concat(axis=1, values=[D_logit,hidden1,hidden2])        #
        #hidden1 = tf.concat(axis=1, values=[prediction1,prediction2,prediction3])        #
print('hidden1',hidden1)
hiddenresu = tf.matmul(hidden1, w_fc)+b_fc 
prediction = tf.nn.softmax(hiddenresu)        
        
        #prediction = tf.nn.softmax(tf.matmul(h_pool2_flat_1, W_fc1)+b_fc1 )
        #+ b_fc2+tf.matmul(h_pool2_flat, W_fc2)+

saver=tf.train.Saver(max_to_keep=3)

theta_D = [w_fc,b_fc]

theta_D1 = [D_W1,D_b1,D_W2,D_b2,D_W3,D_b3]

theta_D2 = [W_conv1,b_conv1,W_conv2,b_conv2,W_conv3,b_conv3,W_conv5,b_conv5,W_fc2,b_fc2]

theta_D3 = [W_conv1_1,b_conv1_1,W_conv2_1,b_conv2_1,W_conv3_1,b_conv3_1,W_conv5_1,b_conv5_1,W_fc1,b_fc1]

# the error between prediction and real data
with tf.name_scope('cross_entropy'):
    cross_entropy = tf.reduce_mean(-tf.reduce_sum(ys * tf.log(prediction), reduction_indices=[1]))       # loss
    #这里用Adam优化器
train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy) #,var_list=theta_D


with tf.name_scope('cross_entropy1'):
    cross_entropy1 = tf.reduce_mean(-tf.reduce_sum(ys * tf.log(prediction1), reduction_indices=[1]))       # loss
    #这里用Adam优化器
train_step1 = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy1)

with tf.name_scope('cross_entropy2'):
    cross_entropy2 = tf.reduce_mean(-tf.reduce_sum(ys * tf.log(prediction2), reduction_indices=[1]))       # loss
    #这里用Adam优化器
train_step2 = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy2)

with tf.name_scope('cross_entropy3'):
    cross_entropy3 = tf.reduce_mean(-tf.reduce_sum(ys * tf.log(prediction3), reduction_indices=[1]))       # loss
    #这里用Adam优化器
train_step3 = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy3)



sess = tf.Session()
# important step
# tf.initialize_all_variables() no long valid from
if int((tf.__version__).split('.')[1]) < 12 and int((tf.__version__).split('.')[0]) < 1:  # tensorflow version < 0.12
    writer = tf.train.SummaryWriter('logs/', sess.graph)
else: # tensorflow version >= 0.12
    #
    writer = tf.summary.FileWriter("logs/", sess.graph)
# 2017-03-02 if using tensorflow >= 0.12
if int((tf.__version__).split('.')[1]) < 12 and int((tf.__version__).split('.')[0]) < 1:
    init = tf.initialize_all_variables()
else:
    init = tf.global_variables_initializer()
sess.run(init)
var_list = tf.trainable_variables()

print('var_list',var_list)





if restorePara:
    saver.restore(sess,checkpoint_dir + 'model_cnn3.ckpt')    
    print('begin w_fc',w_fc)
    print('begin proc')
else:
    k = 0
    for i in range(1000):
        batch_xs, batch_ys = mnist.train.next_batch(300)
        sess.run(train_step1, feed_dict={xs: batch_xs, ys: batch_ys, keep_prob: 0.5})
        sess.run(train_step2, feed_dict={xs: batch_xs, ys: batch_ys, keep_prob: 0.5})
        sess.run(train_step3, feed_dict={xs: batch_xs, ys: batch_ys, keep_prob: 0.5})
        if i % 50 == 0:
            k= int(i/50)%5*1000
            print(compute_accuracy1(
                mnist.test.images[k:k+1000], mnist.test.labels[k:k+1000]))
            print("Times:",i)

    saver.save(sess, checkpoint_dir + 'model_cnn1.ckpt') # global_step=1
    for i in range(1000):
        batch_xs, batch_ys = mnist.train.next_batch(300)
        sess.run(train_step1, feed_dict={xs: batch_xs, ys: batch_ys, keep_prob: 0.5})
        if i % 50 == 0:
            k= int(i/50)%5*1000
            print(compute_accuracy1(
                    mnist.test.images[k:k+1000], mnist.test.labels[k:k+1000]))
    saver.save(sess, checkpoint_dir + 'model_cnn2.ckpt') #, global_step=2
    
k = 0
lastacc =compute_accuracy(
                mnist.test.images[k:k+1000], mnist.test.labels[k:k+1000])

print('first:',lastacc)

batch_xs2_1 =[]
batch_ys2_1 =[]
batch_xs2 = []
batch_ys2 = []
posit = 0
hasRun = False

if (testPara==False):

    for i in range(5000):
        batch_xs, batch_ys = mnist.train.next_batch(200)
        #sess.run(train_step1, feed_dict={xs: batch_xs, ys: batch_ys, keep_prob: 0.5})
        #sess.run(train_step2, feed_dict={xs: batch_xs, ys: batch_ys, keep_prob: 0.5})
        #sess.run(train_step3, feed_dict={xs: batch_xs, ys: batch_ys, keep_prob: 0.5})
        
        sess.run(train_step, feed_dict={xs: batch_xs, ys: batch_ys, keep_prob: 0.5})
        curracc = compute_accuracy(
                    mnist.test.images[k:k+1000], mnist.test.labels[k:k+1000])
        y_pre = sess.run(prediction, feed_dict={xs: batch_xs, ys: batch_ys})    
    
        if i % 50 == 0:
            k= int(i/50)%3*1000
        
            print(compute_accuracy(
                    mnist.test.images[k:k+1000], mnist.test.labels[k:k+1000]))
            print('TImes',i)
            if (curracc>=lastacc):
                saver.save(sess, checkpoint_dir + 'model_cnn3.ckpt') #, global_step=i+2
                lastacc = curracc
                print('save data:')
                print('curracc:',curracc)
        #print('y_pre',y_pre)
    
        #argmaxResu = tf.argmax(y_pre,1)
    
        #print('argmaxResu',argmaxResu)
        #print('argmaxResu',argmaxResu[0])
    
        #for x in argmaxResu:      #b.eval()就得到tensor的数组形式
        #    print(x)
        #print('argmaxResu',argmaxResu)
    
        #argmaxYs = tf.argmax(batch_ys,1)
        #print('argmaxYs',argmaxYs)

    
        #correct_prediction = tf.equal(tf.argmax(y_pre,1), tf.argmax(batch_ys,1))
        #print('correct_prediction shape',correct_prediction.shape.as_list())        
   

        if (i>0):
            
            hasRun = False
            if i%300 ==0:
                print('Get_trainData')
                batch_xs2_1, batch_ys2_1 = Get_trainData()
                batch_xs2 = batch_xs2_1[0:200]
                batch_ys2 = batch_ys2_1[0:200]                  
                sess.run(train_step, feed_dict={xs: batch_xs2, ys: batch_ys2, keep_prob: 0.5})
                posit = 1
                hasRun = True
            if (i>300) and (hasRun==False) and (i%100 == 0):   
                print('run Get_trainData') 
                batch_xs2 = batch_xs2_1[posit*200:(posit+1)*200]
                batch_ys2 = batch_ys2_1[posit*200:(posit+1)*200]    
                if (batch_xs2==[]):
                    print('finish')
                    posit = 0
                    batch_xs2 = batch_xs2_1[posit*200:(posit+1)*200]
                    batch_ys2 = batch_ys2_1[posit*200:(posit+1)*200]    
                
                sess.run(train_step, feed_dict={xs: batch_xs2, ys: batch_ys2, keep_prob: 0.5})
                posit = posit+1
                hasRun = True
                #print('batch_xs2',tf.shape(batch_xs2)[1])
                #print('batch_xs2',tf.shape(batch_ys2)[1])
                #batch_xs2_1, batch_ys2_1 = NextData(batch_xs2_1,batch_ys2_1)
                #print('shape', batch_xs2_1.shape)
                #batch_xs2 = batch_xs2_1[0:200]
                #batch_ys2 = batch_ys2_1[0:200]  
                #print('batch_xs2',batch_xs2)
                #print('batch_ys2',batch_ys2)
                
                #sess.run(train_step, feed_dict={xs: batch_xs2, ys: batch_ys2, keep_prob: 0.5})
                    
                #print('batch_xs2',batch_xs2)
            #dataLen = batch_xs2.shape[0]
            #print('dataLen',dataLen)
            #if (dataLen<=150) and (dataLen>0):
                
            #else:
            #    sess.run(train_step, feed_dict={xs: batch_xs2[0,150], ys: batch_ys2[0,150], keep_prob: 0.5})
            
            
                
                #dataLen = batch_xs2.shape[0]
                #if (dataLen<=150) and (dataLen>0):
            #else:
            #    sess.run(train_step, feed_dict={xs: batch_xs2[0,150], ys: batch_ys2[0,150], keep_prob: 0.5})
 
        
           
print('prog finish')

C:\Anaconda3\envs\gpu-python\lib\site-packages\tensorflow\python\framework\dtypes.py:519: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Anaconda3\envs\gpu-python\lib\site-packages\tensorflow\python\framework\dtypes.py:520: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Anaconda3\envs\gpu-python\lib\site-packages\tensorflow\python\framework\dtypes.py:521: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Anaconda3\envs\gpu-python\lib\site-packages\tensorflow\python\framework\dtypes.py:522: Futu

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data\train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data\train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting MNIST_data\t10k-images-idx3-ubyte.gz
Extracting MNIST_data\t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
h_pool5 Tensor("convolution5/Max_pooling5/MaxPool:0", shape=(?, 4, 4, 256), dtype=float32)
h_pool5 Tensor("convolution5_1/Max_pooling5/MaxPool:0", shape=(?, 4, 4, 256), dtype=float32)
wfc <tf.Variable 'Variable:0' shape=(30, 10) dtype=float32_ref>
hidden1 Tensor("LeakyRelu_3/Maximum:0", shape=(?, 10)